In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from nltk import word_tokenize

reg_dicio = catalog.load('reg_pred_geral_lista')
class_dicio = catalog.load('class_pred_geral_lista')
dicio = reg_dicio.update(class_dicio)

[02/06/24 01:53:14] INFO     Loading data from 'reg_pred_geral_lista' (PartitionedDataset)...   ]8;id=940752;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=641029;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'class_pred_geral_lista' (PartitionedDataset)... ]8;id=766009;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=390724;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py#502\502]8;;\

In [2]:
lista = []

for key, value in reg_dicio.items():
    df = value()
    df['modelo'] = 'regressao_' + key
    lista.append(df)


reg_df = pd.concat(lista)


In [26]:
lista = []

for key, value in class_dicio.items():
    df = value()
    df['modelo'] = 'classificacao_' + key
    lista.append(df)


class_df = pd.concat(lista)

In [27]:
dfzao = pd.concat([reg_df,class_df])
dfzao['abordagem'] = dfzao['modelo'].apply(lambda x:x.split('_')[0])
dfzao['embedding'] =  dfzao['modelo'].apply(lambda x:x.split('_')[1])
dfzao['dimensionalidade'] =  dfzao['modelo'].apply(lambda x:x.split('_')[-1])
dfzao = dfzao.drop(columns = ['modelo'])
dfzao

,index,tema,conjunto,PRED_GERAL,TARGET,abordagem,embedding,dimensionalidade
0,14,2,1,800,840.0,regressao,LSI,128
1,104,39,1,880,840.0,regressao,LSI,128
2,80,13,1,800,520.0,regressao,LSI,128
3,77,2,1,800,920.0,regressao,LSI,128
4,40,54,1,800,640.0,regressao,LSI,128
...,...,...,...,...,...,...,...,...
1521,77,46,1,760,920.0,classificacao,d2v,64
1522,16,43,1,760,680.0,classificacao,d2v,64
1523,54,45,1,920,760.0,classificacao,d2v,64
1524,125,81,1,840,760.0,classificacao,d2v,64


In [28]:
dfzao['dimensionalidade'].unique()

array(['128', '256', '32', '512', '64'], dtype=object)

In [29]:
resultado = dfzao.groupby(['abordagem','embedding','dimensionalidade']).apply(lambda x: cohen_kappa_score(x['PRED_GERAL'],x['TARGET'])).reset_index()

[02/06/24 02:16:16] WARNING  C:\Users\ramon.boucas\AppData\Local\Temp\ipykernel_25612\583045003.py: ]8;id=167203;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\warnings.py\warnings.py]8;;\:]8;id=720962;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\warnings.py#109\109]8;;\
                             1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping                
                             columns. This behavior is deprecated, and in a future version of                      
                             pandas the grouping columns will be excluded from the operation.                      
                             Either pass `include_groups=False` to exclude the groupings or                        
                             explicitly select the grouping columns after groupby to silence this                  
                             warning.                                                                              
                               resultado =                                                                         
                             dfzao.groupby(['abordagem','embedding','dimensionalidade']).apply(lamb                
                             da x: cohen_kappa_score(x['PRED_GERAL'],x['TARGET'])).reset_index()                   
                                                                                                                   

In [30]:
resultado.columns = ['Abordagem','Embedding','Dimensionalidade','Score']
print(resultado.to_latex())

\begin{tabular}{llllr}
\toprule
 & Abordagem & Embedding & Dimensionalidade & Score \\
\midrule
0 & classificacao & LSI & 128 & 0.013114 \\
1 & classificacao & LSI & 256 & 0.003602 \\
2 & classificacao & LSI & 32 & 0.012377 \\
3 & classificacao & LSI & 512 & -0.005016 \\
4 & classificacao & LSI & 64 & -0.003283 \\
5 & classificacao & TF & 128 & -0.006496 \\
6 & classificacao & TF & 256 & -0.002799 \\
7 & classificacao & TF & 32 & -0.002561 \\
8 & classificacao & TF & 512 & 0.008892 \\
9 & classificacao & TF & 64 & -0.007565 \\
10 & classificacao & d2v & 128 & -0.003676 \\
11 & classificacao & d2v & 256 & 0.000965 \\
12 & classificacao & d2v & 32 & -0.004194 \\
13 & classificacao & d2v & 512 & 0.010375 \\
14 & classificacao & d2v & 64 & 0.002954 \\
15 & regressao & LSI & 128 & 0.013114 \\
16 & regressao & LSI & 256 & 0.003602 \\
17 & regressao & LSI & 32 & 0.012377 \\
18 & regressao & LSI & 512 & -0.005016 \\
19 & regressao & LSI & 64 & -0.003283 \\
20 & regressao & TF & 128 & -0.006496

In [12]:
cohen_kappa_score(elef['SOMA_PREDS'],elef['TARGET'])

-0.00017315860876832012

In [13]:
cohen_kappa_score(elef['PRED_GERAL'],elef['TARGET'])

0.0009445858059033974

In [6]:
a = dicio['TF_IDF_32']()
b = dicio['TF_IDF_64']()
c = dicio['d2v_32']()
d = dicio['d2v_64']()

In [8]:
cohen_kappa_score(d['PRED_GERAL'],d['TARGET'])

-0.0008146381831632965

In [16]:
cohen_kappa_score(b['PRED_GERAL'],b['TARGET'])

0.0019551957261074726